In [1]:
import sqlalchemy as sal
import pandas as pd
from os.path import join
import numpy as np
import torch
import os

try:
    from torchts.utils import dataloader
except:
    import sys
    sys.path.append('C:\\Users\\Felix\\Documents\\Spatiotemporal_Data_Repository\\torchTS')
    from torchts.utils import dataloader

In [2]:
dataset = dataloader.seq2seq(root = "C:/Users/Felix/Documents/Spatiotemporal_Data_Repository/torchTS/examples/seq2seq",horizon=12, window=1, features=['avg_speed'],test_ratio = 0.2,train_ratio = 0.7)

(521160, 3)
Number of stations 10
1
2
3
avg_speed


C:\Users\Felix\Documents\Spatiotemporal_Data_Repository\torchTS\torchts\utils\dataloader.py:138: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['timestamp'], dtype='object')
  df_query_cleaned = df_query_raw.fillna(df_query_raw.rolling(window=6,min_periods=1).mean())


4
fin
torch.Size([520920, 24, 1])
torch.Size([520920, 12, 1]) torch.Size([520920, 12, 1])
Total number of samples: 520920
Percentage for train: 70.00
Percentage for val: 10.00
Percentage for test: 20.00
torch.Size([364644, 12, 1]) torch.Size([364644, 12, 1])
torch.Size([52092, 12, 1]) torch.Size([52092, 12, 1])
torch.Size([104184, 12, 1]) torch.Size([104184, 12, 1])
train x:  torch.Size([364644, 12, 1]) y: torch.Size([364644, 12, 1])
val x:  torch.Size([52092, 12, 1]) y: torch.Size([52092, 12, 1])
test x:  torch.Size([104184, 12, 1]) y: torch.Size([104184, 12, 1])


# Define Seq2Seq Model

In [3]:
from torchts.utils import data as DATA
from torchts.nn.models.seq2seq import Encoder, Decoder, Seq2Seq 

import time
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers

import torch.optim as optim

In [4]:
dataset_dir = 'C:/Users/Felix/Documents/Spatiotemporal_Data_Repository/torchTS/examples/seq2seq'

# define hyper-parameters
# learning_rate = 0.01 # this parameter is currently hard coded in the optimizer
dropout_rate = 0.8
num_layers = 1
hidden_dim = 64
input_dim = 1
output_dim = 1
horizon = 12
batch_size = 64


In [5]:
%%time
# load train, val and test NPZ files
data = DATA.load_dataset(dataset_dir, batch_size=batch_size, test_batch_size=batch_size)

data.keys()

Wall time: 196 ms


dict_keys(['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test', 'train_loader', 'val_loader', 'test_loader', 'scaler'])

In [6]:
# instantiate seq2seq model
encoder = Encoder(input_dim, hidden_dim, num_layers, dropout_rate)
decoder = Decoder(output_dim, hidden_dim, num_layers, dropout_rate)

model = Seq2Seq(encoder, decoder, output_dim, horizon, optimizer = torch.optim.Adam)

C:\Users\Felix\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


# Train Model

In [7]:
# define lightning logger and trainner objects
tb_logger = pl_loggers.TensorBoardLogger('logs/')
trainer = Trainer(max_epochs=15, logger=tb_logger, accelerator='gpu', devices=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
data['val_loader']

In [9]:
# train
start = time.time()
trainer.fit(model, data["train_loader"], data["val_loader"])

print("Training time taken %f"%(time.time() - start), flush=True)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 34.3 K
1 | decoder | Decoder | 67.2 K
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.406     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\Felix\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


AttributeError: 'Trainer' object has no attribute 'val_dataloader'

In [ ]:
# see performance for validation dataset
trainer.validate(model, data["val_loader"])

# see performance for test dataset
trainer.test(model, data["test_loader"])

In [ ]:
# save torch pth model to file
torch.save(model, 'saved_model.pth')

# Predict on Test

### Use Checkpoint

In [ ]:
# load model checkpoints and predict from here

# define empty model
encoder = Encoder(input_dim, hidden_dim, num_layers, dropout_rate)
decoder = Decoder(output_dim, hidden_dim, num_layers, dropout_rate)
model = Seq2Seq(encoder, decoder, output_dim, horizon)

# load model object, used local machine because of low RAM memory on Google Colab
model_obj = torch.load('saved_model.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_obj.state_dict())

In [ ]:
%%time

# get predictions for the test dataset
input_tensor = torch.tensor(data['x_test']).float()
target_tensor = torch.tensor(data['y_test']).float()

preds = model(input_tensor, target_tensor).reshape(target_tensor.shape).detach().numpy()

print(preds.shape)

# Visualize Test Predictions

In [ ]:
# # find padding
# batch_size=64

# num_padding_train = (batch_size - len(data["y_train"]) % batch_size) % batch_size
# num_padding_val = (batch_size - len(data["y_val"]) % batch_size) % batch_size
# num_padding_test = (batch_size - len(data["y_test"]) % batch_size) % batch_size

# print('Total Padding No: train {}, val {}, test {}'.format(num_padding_train, num_padding_val, num_padding_test))

In [ ]:
# get float values for unscaling values
avg = float(x_test.mean())
std = float(x_test.std())

print(avg, std)

In [ ]:
def unscale_data(arr, avg, std):
    return arr*std+avg

In [ ]:
# get unscaled y vectors
y_true = unscale_data(data["y_test"], avg, std)
y_pred = unscale_data(preds, avg, std)

print(y_true.shape)
print(y_pred.shape)

In [ ]:
def masked_mae_loss(y_pred, y_true):
    mask = (y_true != 0).float()
    mask /= mask.mean()

    loss = torch.abs(y_pred - y_true)
    loss = loss * mask
    loss[torch.isnan(loss)] = 0

    return loss.mean()

In [ ]:
# find masked MAE
masked_mae = masked_mae_loss(torch.from_numpy(y_pred), torch.from_numpy(y_true))
print('Masked MAE: {:.2f}'.format(masked_mae))

## Single Sequences

In [ ]:
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

plt.figure(figsize=(20,7))
plt.rcParams.update({'font.size': 20})

sequence_index_to_plot = 29607 # specify sequence index 

plt.plot(y_pred[sequence_index_to_plot, :, 0])
plt.plot(y_true[sequence_index_to_plot, :, 0])

plt.legend(['Predictions (Y pred)', 'Actual (Y true)'], loc='upper right', bbox_to_anchor=(1.27, 1))

plt.title('Traffic Sequence Index No. {}, 12 pts Horizon (1hr)'.format(sequence_index_to_plot))
plt.xlabel('Ordered Time Series Points (5 mins intervals)')
plt.ylabel('Average Speed (MPH)')

axes = plt.gca()
# axes.set_xlim([xmin,xmax])
axes.set_ylim([50,80])

## Multiple Sequences with Single Horizon

In [ ]:
plt.figure(figsize=(20,7))
plt.rcParams.update({'font.size': 20})

min_sequence_index_to_plot = 10400 # min limit for x-axis
max_sequence_index_to_plot = 11000 # max limit for x-axis
horizon = 5 # select horizon, integer from 1 to 12

plt.plot(y_pred[horizon+min_sequence_index_to_plot:max_sequence_index_to_plot+horizon, horizon-1, 0])
plt.plot(y_true[min_sequence_index_to_plot:max_sequence_index_to_plot, horizon-1, 0])

plt.legend(['Predictions (Y pred)', 'Actual (Y true)'], loc='upper right', bbox_to_anchor=(1.27, 1))

plt.title("""Traffic Timeseries with Horizon {} (or {} mins), \
Subset from Index No. {} to {}""".format(horizon,
                                                               horizon*5,
                                                               min_sequence_index_to_plot,
                                                               max_sequence_index_to_plot))
plt.xlabel('Ordered Time Series Points (5 mins intervals)')
plt.ylabel('Average Speed (MPH)')

axes = plt.gca()
# axes.set_xlim([xmin,xmax])
axes.set_ylim([0,80])